In [25]:
import os
import pandas as pd
import torch
import torch.nn as nn

from random import sample
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from torch.autograd import Variable 

In [2]:
district ='Dewas'

In [3]:
os.listdir('data')    

['Dewas_NDVI.csv',
 'Dewas_points.csv',
 'Kaithal_NDVI.csv',
 'Kaithal_points.csv',
 'Karnal_NDVI.csv',
 'Karnal_points.csv']

In [4]:
data_points = pd.read_csv(f'data/{district}_points.csv')

In [5]:
data_ndvi = pd.read_csv(f'data/{district}_NDVI.csv')

In [6]:
data = data_points.merge(data_ndvi, left_on='gfid', right_on='gfid')

In [7]:
data.head()

,gfid,state,district,village,lon,lat,wheat,datenum,date,ndvi
0,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,0,2020-10-20,0.186
1,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,1,2020-10-21,0.184
2,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,2,2020-10-22,0.183
3,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,3,2020-10-23,0.182
4,72001,Madhya Pradesh,Dewas,VijayaganjMandi,75.96199,23.218479,0,4,2020-10-24,0.181


In [8]:
data_ndvi.head()

,gfid,datenum,date,ndvi
0,72001,0,2020-10-20,0.186
1,72001,1,2020-10-21,0.184
2,72001,2,2020-10-22,0.183
3,72001,3,2020-10-23,0.182
4,72001,4,2020-10-24,0.181


In [9]:
with open(f'{district}.json') as json_file:
    indices= json.load(json_file)
train = indices['train']
test = indices['test']

In [10]:
features = ['ndvi']
target = ['wheat']

In [11]:
X_train = []
X_test = []
for id_ in train:
    d = data.loc[data['gfid']==id_, features].iloc[:150]
    X_train.append(d)
for id_ in test:
    d = data.loc[data['gfid']==id_, features].iloc[:150]
    X_test.append(d)
X_train = pd.concat(X_train)
X_test = pd.concat(X_test)

In [12]:
h_1 = len(train)
h_2 = len(test)
w = 150
c = len(features)

X_train_tensors = Variable(torch.Tensor(X_train.values.reshape(h_1, w, c)))
X_test_tensors = Variable(torch.Tensor(X_test.values.reshape(h_2, w, c)))

y_train_tensors = Variable(torch.Tensor(data_points.loc[data_points['gfid'].isin(train), 'wheat'].values))
y_test_tensors = Variable(torch.Tensor(data_points.loc[data_points['gfid'].isin(test), 'wheat'].values)) 

y_train_tensors = y_train_tensors.type(torch.LongTensor)
y_test_tensors = y_test_tensors.type(torch.LongTensor)

In [13]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers,
                 seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)
        
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3, inplace=True),
            nn.Linear(in_features=(hidden_size), out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=num_classes),
            nn.Sigmoid()
        )
    def forward(self, x):
        h_0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size)
        out = self.classifier(hn)
        return out

In [14]:
input_size = len(features) #number of features
hidden_size = 32 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 2 #number of output classes 
batch_size = 10

In [15]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1]) #our lstm class

In [16]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(lstm1.parameters(), lr=0.001, momentum=0.9)

In [17]:
num_epochs = 100
for epoch in range(num_epochs):
    
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    outputs = lstm1.forward(X_train_tensors) #forward pass

    # obtain the loss function
    loss = criterion(outputs, y_train_tensors)
    loss.backward() #calculates the loss of the loss function

    optimizer.step() #improve from loss, i.e backprop
    if epoch % 10 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.69223
Epoch: 10, loss: 0.69261
Epoch: 20, loss: 0.69245
Epoch: 30, loss: 0.69237
Epoch: 40, loss: 0.69246
Epoch: 50, loss: 0.69231
Epoch: 60, loss: 0.69253
Epoch: 70, loss: 0.69248
Epoch: 80, loss: 0.69246
Epoch: 90, loss: 0.69234


In [18]:
train_predict = lstm1(X_train_tensors)#forward pass 

In [19]:
train_predict

tensor([[0.5194, 0.4822],
        [0.5169, 0.4827],
        [0.5204, 0.4831],
        [0.5174, 0.4839],
        [0.5193, 0.4820],
        [0.5194, 0.4841],
        [0.5211, 0.4831],
        [0.5214, 0.4800],
        [0.5164, 0.4778],
        [0.5146, 0.4779],
        [0.5207, 0.4824],
        [0.5201, 0.4767],
        [0.5198, 0.4763],
        [0.5223, 0.4827],
        [0.5173, 0.4812],
        [0.5195, 0.4789],
        [0.5139, 0.4795],
        [0.5185, 0.4794],
        [0.5215, 0.4774],
        [0.5202, 0.4803],
        [0.5186, 0.4811],
        [0.5218, 0.4824],
        [0.5181, 0.4760],
        [0.5178, 0.4801],
        [0.5190, 0.4798],
        [0.5187, 0.4783],
        [0.5214, 0.4800],
        [0.5206, 0.4827],
        [0.5201, 0.4797],
        [0.5220, 0.4796],
        [0.5193, 0.4800],
        [0.5200, 0.4779],
        [0.5233, 0.4817],
        [0.5187, 0.4798],
        [0.5170, 0.4804],
        [0.5221, 0.4817],
        [0.5206, 0.4808],
        [0.5169, 0.4809],
        [0.5

In [20]:
train_predict.argmax(dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
print(classification_report(y_train_tensors, torch.argmax(train_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.52      1.00      0.69       157
           1       0.00      0.00      0.00       143

    accuracy                           0.52       300
   macro avg       0.26      0.50      0.34       300
weighted avg       0.27      0.52      0.36       300



C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
test_predict = lstm1.forward(X_test_tensors)

In [23]:
test_predict.argmax(dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])

In [24]:
print(classification_report(y_test_tensors, torch.argmax(test_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.53      1.00      0.70        40
           1       0.00      0.00      0.00        35

    accuracy                           0.53        75
   macro avg       0.27      0.50      0.35        75
weighted avg       0.28      0.53      0.37        75



C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
roc_auc_score(y_test_tensors, torch.argmax(test_predict, dim=1))

0.5